In [1]:
import h2o
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,9 hours 51 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_Aditya_Jain_lwamvf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,223.1 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
N=1000
''' TransactionType from Subscription, Add-on, Upgrade '''

TransactionType = ['Subscription','Subscription','Subscription','Add-on','Add-on','Upgrade','Upgrade','Upgrade']

TransactionType = TransactionType*(round(N/8))

TransactionAmount = np.random.randint(low=200,high=1300,size=N)
''' CBRscore from 0 ...900 '''


CBRscore = np.random.normal(loc=5,scale=2,size=N)

CBRscore[CBRscore<0] = 0

CBRscore[CBRscore>900] = 900

CBRscore = CBRscore.round().astype(int)



d = {'TransactionType': TransactionType, 'TransactionAmount': TransactionAmount,'CBRscore': CBRscore}

df = pd.DataFrame(data=d)

transaction = h2o.H2OFrame(df,destination_frame='transaction')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train, test=transaction.split_frame(
ratios=[0.8],
destination_frames=["transaction_train","transaction_test"],
seed=123
)

In [6]:
print ("%d/%d" % (train.nrows,test.nrows))

788/212


In [7]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [8]:
mGBM_1=H2OGradientBoostingEstimator(model_id="5folds",nfolds=5)

mGBM_1.train(["TransactionType", "TransactionAmount"], "CBRscore", train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:
mGBM_2=H2OGradientBoostingEstimator(model_id="overfit",ntrees=1000,max_depth=10)

mGBM_2.train(["TransactionType", "TransactionAmount"], "CBRscore", train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [10]:
mGBM_1

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  5folds


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 3.7031104454164123
RMSE: 1.9243467581016713
MAE: 1.5508097903377513
RMSLE: 0.39918860766933717
Mean Residual Deviance: 3.7031104454164123

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 4.725624436099063
RMSE: 2.1738501411318727
MAE: 1.7505620407492495
RMSLE: 0.43807809775136175
Mean Residual Deviance: 4.725624436099063
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.7488256,0.0566293,1.8344646,1.7310925,1.7556741,1.607575,1.8153213
mean_residual_deviance,4.7177577,0.2454809,5.063019,4.6102433,4.7536707,4.1155734,5.0462832
mse,4.7177577,0.2454809,5.063019,4.6102433,4.7536707,4.1155734,5.0462832
r2,-0.0758882,0.0287445,-0.0323666,-0.1084698,-0.0781188,-0.0289404,-0.1315455
residual_deviance,4.7177577,0.2454809,5.063019,4.6102433,4.7536707,4.1155734,5.0462832
rmse,2.170527,0.0573152,2.2501152,2.1471477,2.1802914,2.0286877,2.2463934
rmsle,0.4381795,0.0140893,0.4723544,0.4142317,0.4229727,0.4407212,0.4406175


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-12-28 00:17:46,0.386 sec,0.0,2.0972773,1.6537820,4.3985722
,2018-12-28 00:17:46,0.386 sec,1.0,2.0880803,1.6488651,4.3600794
,2018-12-28 00:17:46,0.386 sec,2.0,2.0775266,1.6443618,4.3161167
,2018-12-28 00:17:46,0.386 sec,3.0,2.0703057,1.6409154,4.2861657
,2018-12-28 00:17:46,0.386 sec,4.0,2.0592094,1.6372666,4.2403433
---,---,---,---,---,---,---
,2018-12-28 00:17:46,0.473 sec,46.0,1.9276539,1.5523460,3.7158494
,2018-12-28 00:17:46,0.474 sec,47.0,1.9270696,1.5517760,3.7135973
,2018-12-28 00:17:46,0.476 sec,48.0,1.9261894,1.5516641,3.7102055
,2018-12-28 00:17:46,0.478 sec,49.0,1.9256911,1.5516039,3.7082863



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TransactionAmount,2619.1821289,1.0,0.9080709
TransactionType,265.1544189,0.1012356,0.0919291


In [11]:
mGBM_2

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  overfit


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1.365409217491593
RMSE: 1.168507260350398
MAE: 0.9158952208944988
RMSLE: 0.26158040137942046
Mean Residual Deviance: 1.365409217491593
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-12-28 00:17:49,0.012 sec,0.0,2.0972773,1.6537820,4.3985722
,2018-12-28 00:17:49,0.020 sec,1.0,2.0718698,1.6392338,4.2926444
,2018-12-28 00:17:49,0.022 sec,2.0,2.0495696,1.6286472,4.2007356
,2018-12-28 00:17:49,0.024 sec,3.0,2.0310931,1.6188132,4.1253391
,2018-12-28 00:17:49,0.027 sec,4.0,2.0178313,1.6088901,4.0716431
---,---,---,---,---,---,---
,2018-12-28 00:17:53,4.004 sec,629.0,1.3347008,1.0638289,1.7814263
,2018-12-28 00:17:53,4.012 sec,630.0,1.3340605,1.0632255,1.7797175
,2018-12-28 00:17:53,4.013 sec,631.0,1.3335694,1.0626337,1.7784073
,2018-12-28 00:17:53,4.013 sec,632.0,1.3331851,1.0620870,1.7773825



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
TransactionAmount,11192.8662109,1.0,0.8896420
TransactionType,1388.4481201,0.1240476,0.1103580


In [12]:
perf1=mGBM_1.model_performance(test)
perf1.mae()

1.8057249898216485

In [13]:
perf2=mGBM_2.model_performance(test)
perf2.mae()

1.9073428070460987